In [ ]:
# Full reset of the camera
!echo 'jetson' | sudo -S systemctl restart nvargus-daemon && printf '\n'
# Check device number
!ls -ltrh /dev/video*

# USB Camera (Logitech C270 webcam)
#from jetcam.usb_camera import USBCamera
#camera = USBCamera(width=224, height=224, capture_device=0) # confirm the capture_device number

# CSI Camera (Raspberry Pi Camera Module V2)
from jetcam.csi_camera import CSICamera
camera = CSICamera(width=224, height=224)

camera.running = False
print("camera created")

In [ ]:
from clickable_stream_widget.widget import ClickableImageWidget
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import cv2
import ipywidgets

In [ ]:
stream_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # display saved snapshot
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        
stream_widget.on_msg(save_snapshot)


display(ipywidgets.HBox([stream_widget,snapshot_widget]))

In [ ]:
import traitlets
traitlets.dlink((camera, 'value'), (stream_widget, 'data'))
camera.running = True